TODO before:

    Make sure both are optimised while using the same Best Estimate values of the static uncertainty
    Use those same Best Estimate values for here.


Comparison is actually not completely fair:

    with mpmordm, decisions that are taken later, should also be evaluated in that way.
Describe this als een inschatting die een decision-maker moet maken. Kan het echt niet wachten? Of kan een beetje wachten wel ervoor zorgen dat er meer informatie is en dus een betere beslissing kan worden gemaakt. Daarnaast moet er ook rekening gehouden worden dat sommige beslissingen uitgesteld moeten worden, omdat er bijvoorbeeld niet genoeg resources zijn om meerdere beslissingen tegelijk te implementeren. Juist voor die problemen is MPMORDM een goeie oplossing

Beste Oplossing: Vergelijken met evaluatie voor elke stap en die bij elkaar optellen. Of toch niet?!--> Zie laatste argument

In [1]:
import pickle
import numpy as np
import pandas as pd

from geopy.distance import great_circle

import pygmo as pg

In [2]:
from lib.fl_model_v9 import *

# Load Optimised Policies 

In [3]:
case = "Nepal"

## Load all policies proposed by MORDM Full Uncertainty

In [4]:
# # Store data (serialize)
# with open('{}/MOO/rob_nondom_pols_test1.pickle'.format(case),
#           'wb') as handle:
#     pickle.dump(
#         (rob_nondom_pols),
#         handle,
#         protocol=pickle.HIGHEST_PROTOCOL)
    
with open('{}/MOO/rob_nondom_pols_Full_Uncertainty.pickle'.format(case),
          'rb') as handle:
    Full_Unc_rob_nondom_pols = pickle.load(
        handle)

In [5]:
Full_Unc_rob_nondom_pols.shape

(60,)

## Load all policies proposed by dynamic uncertainty

In [6]:
output_folder = "infov3_4p"

In [7]:
with open('{}/{}/python_objects.pickle'.format(case, output_folder),
          'rb') as handle:
    global_label_tree, EMA_model_dict, global_dfs_var_unc_data, global_fixed_unc, global_policy_tree = pickle.load(
        handle)

In [8]:
p4_labels = global_label_tree['p4']

In [9]:
[global_policy_tree[p4_labels[0]]]

[{'FLO0': 0.0,
  'FLO1': 0.0,
  'FLO2': 0.0,
  'FLO3': 1.0,
  'FLO4': 1.0,
  'FLO5': 0.0,
  'FLO6': 0.0,
  'FLO7': 1.0,
  'FLO8': 0.0,
  'FLO9': 0.0,
  'FLO10': 0.0,
  'FLO11': 1.0,
  'FLO12': 0.0,
  'FLO13': 0.0,
  'FLO14': 0.0,
  'FLO15': 0.0,
  'FLO16': 0.0,
  'FLO17': 0.0,
  'FLO18': 0.0,
  'FLO19': 0.0}]

In [10]:
p4_pols = [global_policy_tree[label] for label in p4_labels]

In [11]:
p4_pols = np.array(p4_pols)

In [12]:
print("P4 after Multi-Period Robust Optimisation contains {} number of policies".format((p4_pols.shape)))

P4 after Multi-Period Robust Optimisation contains (1351,) number of policies


In [13]:
p4_FLs = np.array([[int(key[3:]) for key,value in pol.items() if value == 1] for pol in p4_pols])

In [14]:
p4_FLs = np.unique(p4_FLs,axis=0)

In [15]:
print("Of all {} policies, {} are unique, the rest are duplicates".format(p4_pols.shape[0],p4_FLs.shape[0]))

Of all 1351 policies, 190 are unique, the rest are duplicates


In [16]:
p4_pols_unique = [{"FLO{}".format(i):1 if i in p4_FLs[j] else 0 for i in range(len(p4_pols[0].keys()))} for j in range(len(p4_FLs))]

In [17]:
len(p4_pols_unique)

190

    Compare on hypervolume to see what part of the pareto front is found by mpmordm
    
    If hypervolumes are roughly the same: the question is: what is the possibility that a DM would have chosen from the pareto front?





# Compare on best estimate performance

## Set parameters of best estimates. Should be the same as used for 

In [18]:
Fix_Unc = pd.read_csv("{}/Prepared Input Data/fixed_uncertainties.csv".format(case), index_col=0)

DPs = pd.read_csv("Nepal/Prepared Input Data/DPs.csv")
# DPs.columns

FLs = pd.read_csv("Nepal/Prepared Input Data/FLs.csv")
# FLs.columns

SPs = pd.read_csv("Nepal/Prepared Input Data/SPs.csv")
# SPs.columns

# 0. Instantiate

# General
# Coordinates
# all large cities to demand points
DPY = {"DPY{}".format(i): y for i,y in enumerate(DPs['latitude'])}
DPX = {"DPX{}".format(i): x for i,x in enumerate(DPs['longitude'])}

# all international airports to supply points:
SPY = {"SPY{}".format(i): y for i,y in enumerate(SPs['latitude_deg'])}
SPX = {"SPX{}".format(i): x for i,x in enumerate(SPs['longitude_deg'])}

# Facility locations
FLX = {"FLX{}".format(i): x for i,x in enumerate(FLs['Lon'])}
FLY = {"FLY{}".format(i): y for i,y in enumerate(FLs['Lat'])}


# Population demand points
DPpop = {"DPpop{}".format(i): pop for i,pop in enumerate(DPs['population'])}


In [19]:
Fix_Unc

,lower,best_estimate,upper
var,,,
unit_opening_costs,50.0,100,200
unit_transport_cost,0.5,1,2
FL_operations_cost,20.0,30,50
FL_range,2.0,4,6
demand_per_affected,10.0,20,50


In [20]:
print("Pick right Epicentre, or load disruption data")
print("TODO: Load disruption")

Pick right Epicentre, or load disruption data
TODO: Load disruption


In [21]:
epicentre_name = "RealLocation"

In [22]:
disruptionsFLs = np.loadtxt(
    'Nepal/Prepared Input Data/DisruptionFL{}.txt'.format(epicentre_name))
disruptionsDPs = np.loadtxt(
    'Nepal/Prepared Input Data/DisruptionDP{}.txt'.format(epicentre_name))

In [23]:
# Base real values on Nepal earthquake
# DSR is abbreviation for Disruption

DSRDP = {"DSRDP{}".format(i): disruptionsDPs[i] for i in range(len(DPX))}
DSRFL = {"DSRFL{}".format(i): disruptionsFLs[i] for i in range(len(FLX))}

## Evaluate FullUncPols under best estimates of static uncertainty

In [26]:
Full_Unc_rob_nondom_pols.shape

(60,)

In [27]:
Full_Unc_best_est_results = []
for pol in Full_Unc_rob_nondom_pols:
    Full_Unc_best_est_results.append(
        FL_model(
            unit_opening_costs=Fix_Unc.best_estimate['unit_opening_costs'],
            unit_transport_cost=Fix_Unc.best_estimate['unit_transport_cost'],
            FL_operations_cost=Fix_Unc.best_estimate['FL_operations_cost'],
            demand_per_affected=Fix_Unc.best_estimate['demand_per_affected'],
            graphical_representation=False,
            FL_range=Fix_Unc.best_estimate['FL_range'],
            dist_method='from_table',
            table_dir="{}/Prepared Input Data/routing_table.csv".format(case),
            **SPX,
            **SPY,
            **DPX,
            **DPY,
            **FLX,
            **FLY,
            **pol,
            **DSRDP,
            **DSRFL,
            **DPpop))

In [28]:
Full_Unc_best_est_results = np.array(Full_Unc_best_est_results)

## Evaluate DynamicUncPols under best estimates of static uncertainty

In [29]:
len(p4_pols_unique)

190

In [30]:
Dyn_Unc_best_est_results = []
for pol in p4_pols_unique:
    Dyn_Unc_best_est_results.append(
        FL_model(
            unit_opening_costs=Fix_Unc.best_estimate['unit_opening_costs'],
            unit_transport_cost=Fix_Unc.best_estimate['unit_transport_cost'],
            FL_operations_cost=Fix_Unc.best_estimate['FL_operations_cost'],
            demand_per_affected=Fix_Unc.best_estimate['demand_per_affected'],
            graphical_representation=False,
            FL_range=Fix_Unc.best_estimate['FL_range'],
            dist_method='from_table',
            table_dir="{}/Prepared Input Data/routing_table.csv".format(case),
            **SPX,
            **SPY,
            **DPX,
            **DPY,
            **FLX,
            **FLY,
            **pol,
            **DSRDP,
            **DSRFL,
            **DPpop))

## Check for both if pareto set

In [31]:
import lib.pareto as pareto


### Full Info

In [32]:
pd.DataFrame(Full_Unc_best_est_results).shape

(60, 6)

In [40]:
FullUncPF = np.array(pareto.eps_sort(
    [list(pd.DataFrame(Full_Unc_best_est_results).itertuples(index=True))], [1, 2, 3, 4],
    [1e-9, 1e-9, 1e-9, 1e-9]))

In [41]:
FullUncPF = FullUncPF[:,1:5]

In [42]:
FullUncPF.shape

(52, 4)

### Dyn Uncertainty

In [43]:
pd.DataFrame(Dyn_Unc_best_est_results).shape

(190, 6)

In [44]:
DynUncPF = np.array(pareto.eps_sort(
    [list(pd.DataFrame(Dyn_Unc_best_est_results).itertuples(index=True))], [1, 2, 3, 4],
    [1e-9, 1e-9, 1e-9, 1e-9]))

In [45]:
DynUncPF = DynUncPF[:,1:5]

In [46]:
DynUncPF.shape

(68, 4)

## Calculate hypervolumes of both Pareto Sets

### Normalise 

In [47]:
df_FullUncPF = pd.DataFrame(FullUncPF)

df_FullUncPF['Set'] = "FullUnc"

df_FullUncPF = df_FullUncPF.set_index('Set')

df_FullUncPF.head(2)

,0,1,2,3
Set,,,,
FullUnc,47386.8,21,14245.9,59777.4
FullUnc,210171,14,4799.8,45920.2


In [48]:
df_DynUncPF = pd.DataFrame(DynUncPF)

df_DynUncPF['Set'] = "DynamicUncertainty"

df_DynUncPF = df_DynUncPF.set_index('Set')

df_DynUncPF.head(2)

,0,1,2,3
Set,,,,
DynamicUncertainty,26690.9,24,15967.1,84488
DynamicUncertainty,89526.5,15,11582.4,59777.4


In [49]:
dfs_nonnormalised = pd.concat((df_DynUncPF,df_FullUncPF))

In [50]:
def normalise_df (df):
    normalised_df=(df-df.min())/(df.max()-df.min())
    return normalised_df

In [51]:
dfs_normalised = normalise_df(dfs_nonnormalised)

In [52]:
df_DynUncPF_normalised = dfs_normalised[dfs_normalised.index == 'DynamicUncertainty']

In [53]:
df_FullUncPF_normalised = dfs_normalised[dfs_normalised.index == 'FullUnc']

### Dynamic Uncertainty

In [54]:
df_DynUncPF_normalised.values.astype(float).max(axis=0)

array([1., 1., 1., 1.])

In [71]:
ref_point = [1,1,1,1]

In [72]:
hvDynUnc = pg.hypervolume(df_DynUncPF_normalised.values.astype(float))

In [73]:
hvDynUnc.compute(ref_point)

0.5105584579890514

In [74]:
hvDynUnc.contributions(ref_point).mean()

0.0005060927631837258

### Full Unc

In [75]:
df_FullUncPF_normalised.values.astype(float).max(axis=0)

array([1.        , 1.        , 0.88408934, 1.        ])

In [76]:
hvFullUnc = pg.hypervolume(df_FullUncPF_normalised.values.astype(float))

In [77]:
hvFullUnc.compute(ref_point)

0.5088963866134609

In [78]:
hvFullUnc.contributions(ref_point).mean()

0.0006897220971014236

## Compare hypervolumes of both Pareto Sets

This is a lot larger than the difference between PerfInfo and DynUnc

ToDo:
Compare all three at the same time (normalisation on complete set)

In [81]:
hvDynUnc.compute(ref_point) - hvFullUnc.compute(ref_point)

0.0016620713755904903

# Compare on Robustness performance